In [1]:
import liana as li
import scanpy as sc
from anndata import AnnData
import pandas as pd
from liana.mt import rank_aggregate
from liana.method import singlecellsignalr, connectome, cellphonedb, natmi, logfc, cellchat, geometric_mean
from pycirclize import Circos

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# loading original anndata
pvat_8weeks_hf_female = sc.read_h5ad('../../exported_data/pvat_8weeks_hf_female.h5ad')
pvat_8weeks_hf_female

AnnData object with n_obs × n_vars = 6314 × 20743
    obs: 'sample_id', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'outlier', 'mt_outlier', 'n_genes', 'scDblFinder_score', 'scDblFinder_class', 'Sample', 'tissue', 'diet', 'time', 'sex', 'batch', 'sample_type', 'rat_id', 'weight_3W', 'weight_10W', 'weight_17W', 'weight_24W', 'MAP_7W', 'systolic_bp_7W', 'MAP_9W', 'systolic_bp_9W', 'MAP_16W', 'systolic_bp_16W', 'MAP_24W', 'systolic_bp_24W', 'doublet', 'celltype', 'celltype_broad'
    var: 'gene_ids', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells'
    uns: 'cellchat_interaction_table_count', 'cellchat_res', 'cellchat_res_filtered', 

In [3]:
pvat_8weeks_hf_female.obs['celltype_broad'].unique().tolist()

['Adipocytes',
 'Fibroblasts',
 'Endothelial_Cells',
 'Immune_Cells',
 'Pericytes',
 'Mesothelial_Cells',
 'SMCs',
 'Neuronal_Cells']

In [4]:
# List available methods in lliana framework
li.mt.show_methods()

,Method Name,Magnitude Score,Specificity Score,Reference
0,CellPhoneDB,lr_means,cellphone_pvals,"Efremova, M., Vento-Tormo, M., Teichmann, S.A...."
0,Connectome,expr_prod,scaled_weight,"Raredon, M.S.B., Yang, J., Garritano, J., Wang..."
0,log2FC,None,lr_logfc,"Dimitrov, D., Türei, D., Garrido-Rodriguez, M...."
0,NATMI,expr_prod,spec_weight,"Hou, R., Denisenko, E., Ong, H.T., Ramilowski,..."
0,SingleCellSignalR,lrscore,None,"Cabello-Aguilar, S., Alame, M., Kon-Sun-Tack, ..."
0,CellChat,lr_probs,cellchat_pvals,"Jin, S., Guerrero-Juarez, C.F., Zhang, L., Cha..."
0,Rank_Aggregate,magnitude_rank,specificity_rank,"Dimitrov, D., Türei, D., Garrido-Rodriguez, M...."
0,Geometric Mean,lr_gmeans,gmean_pvals,CellPhoneDBv2's permutation approach applied t...


In [5]:
pvat_8weeks_hf_female.raw = pvat_8weeks_hf_female
pvat_8weeks_hf_female.raw.X

<6314x20743 sparse matrix of type '<class 'numpy.float64'>'
	with 11552260 stored elements in Compressed Sparse Row format>

In [6]:
# Run CellPhoneDB analysis on PVAT data
cellphonedb(pvat_8weeks_hf_female, groupby='celltype_broad', expr_prop=0.1, resource_name='mouseconsensus', verbose=True, key_added='cellphonedb_res')

Using `.raw`!
/Users/pandavis/miniconda3/lib/python3.11/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
1598 features of mat are empty, they will be removed.
Make sure that normalized counts are passed!
/Users/pandavis/miniconda3/lib/python3.11/site-packages/liana/method/_pipe_utils/_pre.py:148: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
['Cyp2c6_v1-ps2'] contain `_`. Consider replacing those!
Using resource `mouseconsensus`.
0.21 of entities in the resource are missing from the data.


Generating ligand-receptor stats for 6314 samples and 19145 features


100%|███████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 307.07it/s]


In [7]:
ligand_receptor_pvat = pvat_8weeks_hf_female.uns['cellphonedb_res']
ligand_receptor_pvat.head()

,ligand,ligand_complex,ligand_means,ligand_props,receptor,receptor_complex,receptor_means,receptor_props,source,target,lr_means,cellphone_pvals
2955,Slit3,Slit3,56.454548,1.000000,Robo4,Robo4,0.336621,0.254930,SMCs,Endothelial_Cells,28.395584,0.0
5861,Efna5,Efna5,27.131390,0.968857,Ephb1,Ephb1,1.401868,0.563863,Adipocytes,Mesothelial_Cells,14.266629,0.0
3175,Efna5,Efna5,27.131390,0.968857,Ephb2,Ephb2,0.694235,0.280702,Adipocytes,Fibroblasts,13.912812,0.0
1466,Efna5,Efna5,27.131390,0.968857,Epha4,Epha4,0.514085,0.260563,Adipocytes,Endothelial_Cells,13.822738,0.0
7629,Efna5,Efna5,27.131390,0.968857,Ephb2,Ephb2,0.357143,0.214286,Adipocytes,Neuronal_Cells,13.744267,0.0


In [8]:
ligand_receptor_string = (ligand_receptor_pvat.loc[:,'ligand_complex'].str.contains('Col')) & (ligand_receptor_pvat.loc[:,'receptor_complex'].str.contains('Itg'))
cell_source_string = ligand_receptor_pvat.loc[:,'source'] == 'Fibroblasts'
cell_target_string = ligand_receptor_pvat.loc[:,'target'] == 'Adipocytes'
cell_pvals = ligand_receptor_pvat.loc[:,'cellphone_pvals'] <= 0.05

ligand_receptor_pvat[ligand_receptor_string & cell_source_string & cell_target_string & cell_pvals]

,ligand,ligand_complex,ligand_means,ligand_props,receptor,receptor_complex,receptor_means,receptor_props,source,target,lr_means,cellphone_pvals
319,Col4a1,Col4a1,7.620283,0.778195,Itgb1,Itga1_Itgb1,0.896812,0.477963,Fibroblasts,Adipocytes,4.258547,0.0
325,Col4a2,Col4a2,7.100234,0.753133,Itgb1,Itga1_Itgb1,0.896812,0.477963,Fibroblasts,Adipocytes,3.998523,0.0
415,Col4a2,Col4a2,7.100234,0.753133,Itgb5,Itgb5,0.345736,0.225917,Fibroblasts,Adipocytes,3.722985,0.0
353,Col3a1,Col3a1,5.878432,0.878446,Itgb1,Itga1_Itgb1,0.896812,0.477963,Fibroblasts,Adipocytes,3.387622,0.0
347,Col5a3,Col5a3,5.499985,0.820802,Itgb1,Itga1_Itgb1,0.896812,0.477963,Fibroblasts,Adipocytes,3.198398,0.0
331,Col6a3,Col6a3,3.900993,0.766917,Itgb1,Itga1_Itgb1,0.896812,0.477963,Fibroblasts,Adipocytes,2.398902,0.0
329,Col6a2,Col6a2,2.986208,0.770677,Itgb1,Itga1_Itgb1,0.896812,0.477963,Fibroblasts,Adipocytes,1.941510,0.0
349,Col5a2,Col5a2,2.798239,0.733083,Itgb1,Itga1_Itgb1,0.896812,0.477963,Fibroblasts,Adipocytes,1.847525,0.0
343,Col15a1,Col15a1,2.701748,0.627820,Itgb1,Itga1_Itgb1,0.896812,0.477963,Fibroblasts,Adipocytes,1.799280,0.0
315,Col6a5,Col6a5,2.650372,0.447368,Itgb1,Itga1_Itgb1,0.896812,0.477963,Fibroblasts,Adipocytes,1.773592,0.0


In [9]:
ligand_receptor_pvat.to_csv("pvat_8weeks_hf_female_cellphonedb.csv")
pvat_8weeks_hf_female.write("../../exported_data/pvat_8weeks_hf_female.h5ad")